In [2]:
import igviz as ig

from owlready2 import *
import urllib.request
import fastobo

import pandas as pd

import networkx as nx

from dash import Dash, html, dcc, Input, Output
import plotly.express as px

from jupyter_dash import JupyterDash

In [4]:
onto = get_ontology("doid.owl").load()
disease_name = {i.get_name(i).replace('_',':'): i.label[0] for i in list(onto.classes())}

In [5]:
with urllib.request.urlopen("https://raw.githubusercontent.com/DiseaseOntology/HumanDiseaseOntology/main/src/ontology/doid.obo") as response:
    pato = fastobo.load(response)

In [6]:
knowledge_graph = nx.DiGraph()

for frame in pato:
    if isinstance(frame, fastobo.term.TermFrame):
        knowledge_graph.add_node(str(frame.id))
        for clause in frame:
            if isinstance(clause, fastobo.term.IsAClause):
                knowledge_graph.add_edge(str(frame.id), str(clause.term))

In [12]:
def disease_fig(id1 = "DOID:9745", id2 = "DOID:2739"):

    superclass_nodes = nx.descendants(knowledge_graph, id1)
    superclass_nodes.add(id1)
    super_graph_1 = knowledge_graph.subgraph(superclass_nodes)
    superclass_nodes = nx.descendants(knowledge_graph, id2)
    superclass_nodes.add(id2)
    super_graph_2 = knowledge_graph.subgraph(superclass_nodes)


    G = nx.compose(super_graph_1,super_graph_2)

    pos = nx.nx_agraph.graphviz_layout(G, prog="dot")
    pos = {i: (pos[i][0],-pos[i][1]) for i in pos}

    nx.set_node_attributes(G, pos, "pos")
    nx.set_node_attributes(G, {node: 'red' if node in [id1,id2] else 'blue' for node in G.nodes()}, "seed")
    nx.set_node_attributes(G, disease_name, "name")
    nx.set_node_attributes(G, {node: node for node in G.nodes()}, "id")

    nx.set_edge_attributes(G, 'is_a', 'label')

    color_list = ['red','blue']

    fig = ig.plot(G,
                  color_method = 'seed',
                  size_method = 'static',
                  edge_label = 'label',
                  edge_label_position = 'top left',
                  node_label = 'id',
                  node_label_position = 'middle left',
                  node_text = ['name'])

    return fig

In [29]:
app = JupyterDash(__name__)

app.layout = html.Div(children=[
     html.H6("Type Disease 1 DO ID"),
    html.Div([
        "Input: ",
        dcc.Input(id='disease_1', value='DOID:9745', type='text')
    ]),
    html.Br(),
    html.Div(id='my-output-1'),
    html.H6("Type Disease 2 DO ID"),
    html.Div([
        "Input: ",
        dcc.Input(id='disease_2', value='DOID:90', type='text')
    ]),
    html.Br(),
    html.Div(id='my-output-2'),

    dcc.Graph(figure=disease_fig(),id='disease_graph')
])


@app.callback(
    Output('disease_graph', 'figure'),
    Input('disease_1', 'value'),
    Input('disease_2','value')
)

def update_figure(id1,id2):
    
    fig = disease_fig(id1,id2)
    
    fig.update_layout(transition_duration=1000)
    
    return fig

In [30]:
if __name__ =='__main__':
    app.run_server(mode="external")

Dash app running on http://127.0.0.1:8050/
